In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/news-category-trainingdata-final/news_category_trainingdata.json


# **1) Download KTrain**

In [ ]:
try:
  import ktrain
except:
  !pip install ktrain
  os.kill(os.getpid(), 9)
import ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 49.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 52.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 17.0 MB/s eta 0:00:00
  Created wheel 

# **2) Import required libraries and load dataset**

In [1]:
import numpy as np
import pandas as pd
import ktrain
reviews = pd.read_json("/kaggle/input/news-category-trainingdata-final/news_category_trainingdata.json")
reviews.head()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [2]:
reviews.describe()

,date
count,200853
mean,2015-02-06 05:55:12.542008576
min,2012-01-28 00:00:00
25%,2013-07-18 00:00:00
50%,2015-01-25 00:00:00
75%,2016-08-19 00:00:00
max,2018-05-26 00:00:00


# **3) Create column with combined texts "headline" and "short description" in prep for model creation**

In [3]:
reviews['combined_text'] = reviews['headline'] + ' ' + reviews['short_description']
reviews[reviews['category'].str.contains("HEALTHY LIVING")]

,category,headline,authors,link,short_description,date,combined_text
7578,HEALTHY LIVING,To The People Who Say ‘I’m Tired’ When Someone...,"The Mighty, ContributorWe face disability, dis...",https://www.huffingtonpost.com/entry/to-the-pe...,"When you feel like this, it’s important to kno...",2018-01-16,To The People Who Say ‘I’m Tired’ When Someone...
7693,HEALTHY LIVING,Eating Shake Shack Made Me Feel Healthier Than...,"Colleen Werner, ContributorCampus Editor-at-Large",https://www.huffingtonpost.com/entry/eating-sh...,I can vividly remember the first time I felt f...,2018-01-12,Eating Shake Shack Made Me Feel Healthier Than...
7747,HEALTHY LIVING,How To Stay Updated On The News Without Losing...,Lindsay Holmes,https://www.huffingtonpost.com/entry/anxiety-f...,Because it's only becoming more of a struggle.,2018-01-12,How To Stay Updated On The News Without Losing...
7927,HEALTHY LIVING,27 Perfect Tweets About Whole30 That Will Make...,Lindsay Holmes,https://www.huffingtonpost.com/entry/tweets-ab...,"""The only Whole30 I want to participate in is ...",2018-01-10,27 Perfect Tweets About Whole30 That Will Make...
7934,HEALTHY LIVING,The Real Reason Your Hands Are Always Cold,"Refinery29, ContributorThe #1 new-media brand ...",https://www.huffingtonpost.com/entry/the-real-...,"Essentially, your hands are kept warm thanks t...",2018-01-10,The Real Reason Your Hands Are Always Cold Ess...
...,...,...,...,...,...,...,...
124913,HEALTHY LIVING,Why You Need Both a 'Bouncer' and a 'Bartender...,"Elizabeth Grace Saunders, ContributorFounder, ...",https://www.huffingtonpost.com/entry/happy-hea...,Instead of judging whether you made the right ...,2014-04-18,Why You Need Both a 'Bouncer' and a 'Bartender...
124914,HEALTHY LIVING,How Video Games Can Improve Dialogue on Mental...,"Mona Shattell, Contributornurse researcher",https://www.huffingtonpost.com/entry/mental-il...,While there are strong arguments for the games...,2014-04-18,How Video Games Can Improve Dialogue on Mental...
124925,HEALTHY LIVING,Wake-Up Calls Inspired My Change From Overdriv...,"Jane Shure, ContributorLeadership Coach, Psych...",https://www.huffingtonpost.com/entry/wake-up-c...,My wake-up call marching orders were clear: No...,2014-04-18,Wake-Up Calls Inspired My Change From Overdriv...
124950,HEALTHY LIVING,Loving a Narcissist Without Losing Yourself,"Nancy Colier, ContributorPsychotherapist, inte...",https://www.huffingtonpost.com/entry/narcissis...,It is very difficult for some people to see an...,2014-04-18,Loving a Narcissist Without Losing Yourself It...


In [4]:
reviews['healthy'] = np.where((reviews['category'] == 'HEALTHY LIVING'), 1, 0)
reviews['healthy'].describe()

count    200853.000000
mean          0.033328
std           0.179492
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: healthy, dtype: float64

# **4) Split dataset with a 20/80 distribution for model training and validation purposes**

In [5]:
#splitting data 20/80 instead of 50/50
# Calculate the sample amounts for each subset
sample_amount_healthy = int(len(reviews[reviews["healthy"] == 1]) * 0.2)
sample_amount_not_healthy = int(len(reviews[reviews["healthy"] == 0]) * 0.8)

# Sample the subsets with the updated ratios
healthy = reviews[reviews['healthy'] == 1].sample(n=sample_amount_healthy)
not_healthy = reviews[reviews['healthy'] == 0].sample(n=sample_amount_not_healthy)

In [6]:
review_sample = pd.concat([healthy,not_healthy])
review_sample.describe()

,date,healthy
count,156665,156665.000000
mean,2015-01-30 17:50:14.449941248,0.008541
min,2012-01-28 00:00:00,0.000000
25%,2013-07-05 00:00:00,0.000000
50%,2015-01-17 00:00:00,0.000000
75%,2016-08-18 00:00:00,0.000000
max,2018-05-26 00:00:00,1.000000
std,NaN,0.092020


In [7]:
target_names = ['NOT HEALTHY LIVING','HEALTHY LIVING']

# **5) Distilbert Model Creation**

In [8]:
train, val, preprocess = ktrain.text.texts_from_df(
    review_sample,
    "combined_text",
    label_columns=["healthy"],
    val_df=None,
    max_features=20000,
    maxlen=512,
    val_pct=0.1,
    ngram_range=1,
    preprocess_mode="distilbert",
    verbose=1
)

['not_healthy', 'healthy']
        not_healthy  healthy
161480          1.0      0.0
167343          1.0      0.0
71781           1.0      0.0
15151           1.0      0.0
160262          1.0      0.0
['not_healthy', 'healthy']
        not_healthy  healthy
33064           1.0      0.0
35567           1.0      0.0
45494           1.0      0.0
163752          1.0      0.0
98663           1.0      0.0


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

preprocessing train...
language: en
train sequence lengths:
	mean : 29
	95percentile : 55
	99percentile : 66


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 29
	95percentile : 55
	99percentile : 66


# **6) Distilbert model training with 2 epochs and a batch size of 16**

In [9]:
model_distilbert = preprocess.get_classifier()
learner_distilbert = ktrain.get_learner(model_distilbert, train_data=train, val_data=val, batch_size=16)
learner_distilbert.fit_onecycle(2e-5, 2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
8813/8813 [==============================] - 4773s 537ms/step - loss: 0.0461 - accuracy: 0.9902 - val_loss: 0.0398 - val_accuracy: 0.9907
Epoch 2/2
8813/8813 [==============================] - 4731s 537ms/step - loss: 0.0273 - accuracy: 0.9923 - val_loss: 0.0370 - val_accuracy: 0.9910


In [13]:
learner_distilbert.lr_find(max_epochs=2)

simulating training for different learning rates... this may take a few moments...


In [11]:
learner_distilbert.lr_plot()

ValueError: Please call lr_find first.

In [15]:
history=learner_distilbert.autofit(
    1e-4,
    checkpoint_folder='checkpoint',
    epochs=5,
    early_stopping=True
)



begin training using triangular learning rate policy with max lr of 0.0001...
Epoch 1/5
2119/8813 [======>.......................] - ETA: 58:55 - loss: 0.0285 - accuracy: 0.9919

KeyboardInterrupt: 

# **7) Distilbert model predictor generator**

In [ ]:
predictor = ktrain.get_predictor(learner_distilbert.model, preproc=preprocess)

# **8) Distilbert model validation**

In [18]:
validation = learner_distilbert.validate(val_data=val, print_report=True)

490/490 [==============================] - 78s 155ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     15519
           1       0.60      0.02      0.04       148

    accuracy                           0.99     15667
   macro avg       0.80      0.51      0.52     15667
weighted avg       0.99      0.99      0.99     15667



# **9) Bert model creation**

In [7]:
# Code using BERT model
train, val, preprocess = ktrain.text.texts_from_df(
    review_sample,
    "combined_text",
    label_columns=["healthy"],
    val_df=None,
    max_features=20000,
    maxlen=512,
    val_pct=0.1,
    ngram_range=1,
    preprocess_mode="bert",
    verbose=1
)

['not_healthy', 'healthy']
        not_healthy  healthy
55625           1.0      0.0
128779          1.0      0.0
109209          1.0      0.0
121440          1.0      0.0
195337          1.0      0.0
['not_healthy', 'healthy']
        not_healthy  healthy
81936           1.0      0.0
156173          1.0      0.0
110129          1.0      0.0
107685          1.0      0.0
81407           1.0      0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


# **10) Bert model training with 2 epochs and a batch size of 4**

In [12]:
from ktrain import text
model_bert = text.text_classifier('bert', train, preproc=preprocess)
learner_bert = ktrain.get_learner(model_bert, train_data=train, val_data=val, batch_size=4)
learner_bert.fit_onecycle(2e-5, 2)


Is Multi-Label? False
maxlen is 512


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


done.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
35250/35250 [==============================] - 12121s 343ms/step - loss: 0.0408 - accuracy: 0.9912 - val_loss: 0.0393 - val_accuracy: 0.9916
Epoch 2/2
35250/35250 [==============================] - ETA: 0s - loss: 0.0336 - accuracy: 0.9916

KeyboardInterrupt: 

# **11) Bert model predictor generator**

In [21]:
bert_predictor = ktrain.get_predictor(learner_bert.model, preproc=preprocess)

# **12) Bert model validation**

In [16]:
validation = learner_bert.validate(val_data=val, print_report=True)

490/490 [==============================] - 364s 731ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     15535
           1       0.62      0.11      0.19       132

    accuracy                           0.99     15667
   macro avg       0.81      0.56      0.59     15667
weighted avg       0.99      0.99      0.99     15667



In [24]:
!pip3 install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1

In [19]:
test_docs = [
'Stress May Be Your Heart’s Worst Enemy Psychological stress activates the fear center in the brain, setting into motion a cascade of reactions that can lead to heart attacks and strokes.',
'Exercising to Slim Down? Try Getting Bigger. It’s high time for women to reclaim the real strength behind exercise.',
'What Are Your Food Resolutions for the New Year? Join us for the Eat Well Challenge starting in January.',
'Why We All Need to Have More Fun. Prioritizing fun may feel impossible right now. But this four-step plan will help you rediscover how to feel more alive.',
'Cuomo Will Not Be Prosecuted in Groping Case, Albany D.A. Says. The district attorney described the woman who said former Gov. Andrew Cuomo had groped her as “credible,” but added that proving her allegation would be difficult.',
'A Film Captures Jewish Life in a Polish Town Before the Nazis Arrived. A documentary based on a home movie shot by an American in 1938 provides a look at the vibrancy of a Jewish community in Europe just before the Holocaust.']

In [22]:
for i, text in enumerate(test_docs):
  probs = bert_predictor.predict(text, return_proba=True)
  print("---------------------------")
  print('The probability this is healthy is %s' % probs[1])
  print(text)

---------------------------
The probability this is healthy is 0.10536711
Stress May Be Your Heart’s Worst Enemy Psychological stress activates the fear center in the brain, setting into motion a cascade of reactions that can lead to heart attacks and strokes.
---------------------------
The probability this is healthy is 0.016446155
Exercising to Slim Down? Try Getting Bigger. It’s high time for women to reclaim the real strength behind exercise.
---------------------------
The probability this is healthy is 0.041128434
What Are Your Food Resolutions for the New Year? Join us for the Eat Well Challenge starting in January.
---------------------------
The probability this is healthy is 0.011630355
Why We All Need to Have More Fun. Prioritizing fun may feel impossible right now. But this four-step plan will help you rediscover how to feel more alive.
---------------------------
The probability this is healthy is 4.309325e-05
Cuomo Will Not Be Prosecuted in Groping Case, Albany D.A. Says